In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# ABOUT HABERNMAN'S SURVIVAL DATA SET
The dataset contains cases from a study that was conducted between 1958 and 1970 at the University of Chicago's Billings Hospital on the survival of patients who had undergone surgery for breast cancer.
***
### ATTRIBUTES:

**Age:** Age of patient at time of operation (numerical) .  

**Op_Year:** Patient's year of operation (year - 1900, numerical) 

**Axil_nodes: **Number of positive axillary nodes detected (numerical) 

**Surv_status: **Survival status (class attribute) 

  ----------------**1** = the patient survived 5 years or longer 

----------------**2** = the patient died within 5 year
                    




In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

hab = pd.read_csv("../input/haberman.csv",names = ['Age', 'Operation_year', 'Positive_axillary_nodes', 'Survival_status'])
hab.info()

**OBSERVATIONS:**
* No of instances = 306
* There is no missing values in all of the three attributes.
* All the attribute's values are in integer.

## Higher Level Statics

In [ ]:
features = list(hab.columns[:-1])
classes = hab.Survival_status.value_counts()
print("No. of points:",len(hab))
print("No. of features:",len(features),' namely',','.join(features))
print("No. of classes:",len(classes))
print("No. of points per classes:\n",classes)

**Observation:**

Data is having imbalanced classes.

### STATISTICAL INFORMATION

In [ ]:
hab.describe()

**Observations:**
* Average age of the patient is 52 yrs and the range lies between 30 to 80 out of which: 

   * About 25 % of the patients have the age around 30-44.
   * About 75% of the patients have the age around 30-60.

* Max number of positive auxillary nodes detected is 52.
     * About 25% of the patients have no positive axillary lymph nodes.
     * About 75 % of the patients have around 0-4 positive axillary lymph nodes.           
* About 75% of the patients have survived 5 years or more than it.
* About 25% of the patients have survived less than 5 years.


In [ ]:
'''f = plt.figure(figsize=(48,28))
hab1 = hab.groupby(['Age','Survival_status']).size()
hab1.unstack().plot(kind='bar',figsize=(28,10),fontsize=20,label=['survived 5 years or longer','died within 5 year'])
plt.show()'''


### How many survived 5 years or more than it?

In [ ]:
f,ax = plt.subplots(1,2,figsize=(18,8))
hab['Survival_status'].value_counts().plot(kind='pie',explode=[0,0.1],autopct='%1.1f%%', ax=ax[0],labels=['survived 5 years or longer','died within 5 year'],fontsize=15, shadow=True)
ax[0].set_title('Survival Status',fontsize=25)
ax[0].set_ylabel('')
sns.countplot(hab['Survival_status'], ax=ax[1])
ax[1].set_title('Survival Status',fontsize=25)
ax[1].set_xticklabels(labels=['survived 5 years or longer','died within 5 year'],fontsize=15)
for p in ax[1].patches:
    ax[1].annotate('{:d}'.format(p.get_height()), (p.get_x()+0.35, p.get_height()+3),fontsize=15)
plt.show()

**Observations:**
* Out of **306** patients **225** patients *survived 5 years or longer* than it i.e. About **73.5%** of the total patients lived for *5 years or longer than that*.
* **81** patients *died within 5 years* i.e **26.5%** of the total patients *died within 5 years*.

 ## Objective
***To predict the patient survival who had undergone surgery for breast cancer.***

## Univariate Analysis
 It provides summary statistics for each field in the raw data set. 
 Example:  **PDF ,CDF ,Box plot ,Violin plot**.
 It does not deal with cause or relationships and the main purpose of the analysis is to describe the data and find patterns  that exist within it.
 


### PDF
* A histogram can be used to illustrate the shape, or the distribution, of data. It plot the frequency of score occurrences in a continuous data set that has been divided into classes, called bins. The height of the bins either represents counts, or it represents proportions.
* A probability density function (PDF) is the continuous version of the histogram with densities (consider infinitesimal small bin widths).
* It specifies how the probability density is distributed over the range of values that a random variable can take.

In [ ]:
for idx, feature in enumerate(list(hab.columns)[:-1]):
    g=sns.FacetGrid(hab, hue="Survival_status", size=6) \
       .map(sns.distplot, feature) \
       .add_legend()
    new_labels = ['survived 5 years or longer','died within 5 year'] 
    for t, l in zip(g._legend.texts, new_labels): t.set_text(l) 
    plt.ylabel("Density")
    plt.title("Histogram of "+feature)
    plt.show()

**Observation:**
As we can the Survival status classes are overlapping massively in each of the Histogram of feature. But from the last one we can conclude that around 56-58% people who survived 5 years or longer had 0-5 Positive axillary nodes.

### CDF
The cumulative distribution function (CDF) of a real-valued random variable X is the probability that X will take a value less than or equal to x.

In [ ]:
class_1 = hab.loc[hab["Survival_status"] == 1];
class_2 = hab.loc[hab["Survival_status"] == 2];


In [ ]:
for idx, feature in enumerate(list(hab.columns)[:-1]):
    plt.figure(figsize=(10,5))
    
    label = ['PDF of those who survived 5 years or longer','CDF of those who survived 5 years or longer','PDF of those who died within 5 year','CDF of those who died within 5 year']
    
    counts, bin_edges = np.histogram(class_1[feature], bins=10,density = True)
    pdf = counts/(sum(counts))
    print("PDF: ",pdf);
    print("Bin edges: ",bin_edges);
    
    cdf = np.cumsum(pdf)
    plt.plot(bin_edges[1:],pdf)
    plt.plot(bin_edges[1:], cdf)
    
    counts, bin_edges = np.histogram(class_2[feature], bins=10,density = True)
    pdf = counts/(sum(counts))
    print("PDF: ",pdf);
    print("Bin edges: ",bin_edges);  

    cdf = np.cumsum(pdf)
    plt.plot(bin_edges[1:],pdf)
    plt.plot(bin_edges[1:], cdf)
    
    plt.ylabel("Density")
    plt.title("PDF and CDF of "+feature)
    
    plt.legend(label)
    
    plt.show()


**Observation**
*  About 16% who survived 5 years or more than it are aged around 35-38.
* About 12% who died within 5 years had  47-52 +ve axillary nodes 

### BOX PLOT
* Efficient way for presenting data, especially when it comes to comparing multiple groups thereof.
* We can mark-off the five-number summary of a data set (minimum, 25th percentile, median, 75th percentile, maximum). 
* The box contains 50% of the data, and the upper edge of the box represents the 75th percentile, while the lower edge is the 25th percentile.
* The median is represented by a line drawn in the middle of the box. 
* The interquartile range(IQR) is the difference between the upper quartile and the lower quartile.
* Useful because it is less influenced by extreme values as it limits the range to the middle 50% of the values.

In [ ]:
labels = ['survived 5 years or longer','died within 5 year']
fig, axes = plt.subplots(1, 3, figsize=(25, 5))

for idx, feature in enumerate(list(hab.columns)[:-1]):

    ax=sns.boxplot(x='Survival_status',y=feature, data=hab,hue = "Survival_status",ax=axes[idx])
    medians = hab.groupby(['Survival_status'])[feature].median().values
    median_labels = [str(np.round(s, 2)) for s in medians]
    pos = range(len(medians))
    for tick,label in zip(pos,ax.get_xticklabels()):
        ax.text(pos[tick]-0.1, medians[tick]+0.2, median_labels[tick], 
            horizontalalignment='center', size='x-large', color='b', weight='semibold')
    h, l = ax.get_legend_handles_labels()
    ax.legend(h, labels, title="Survival Status")
    ax.set_title("Box plot between {} and Survival_status".format(feature))
    #plt.legend(label)

plt.show()

**Observations**
* The patients who operated after 1965 had chances to survive and below 1960 had chances to die.
* Those who died within 5 years had atleast 4 positive axillary nodes.

### VIOLIN PLOT
* A Violin Plot is used to visualise the distribution of the data and its probability density.
* It's a combination of a Box Plot and a Density Plot.
* The thick black bar in the centre represents the interquartile range, the thin black line extended from it represents the 95% confidence intervals, and the white dot is the median.


In [ ]:
labels = ['survived 5 years or longer','died within 5 year']
fig, axes = plt.subplots(1, 3, figsize=(25, 5))

for idx, feature in enumerate(list(hab.columns)[:-1]):

    ax=sns.violinplot(x='Survival_status',y=feature, data=hab,hue = "Survival_status",ax=axes[idx])
    medians = hab.groupby(['Survival_status'])[feature].median().values
    median_labels = [str(np.round(s, 2)) for s in medians]
    pos = range(len(medians))
    for tick,label in zip(pos,ax.get_xticklabels()):
        ax.text(pos[tick]-0.1, medians[tick]+0.2, median_labels[tick], 
            horizontalalignment='center', size='x-large', color='b', weight='semibold')
    h, l = ax.get_legend_handles_labels()
    ax.legend(h, labels, title="Survival Status")
    ax.set_title("Violin plot between {} and Survival_status".format(feature))
    #plt.legend(label)

plt.show()

## Bi-variate analysis
* Analysis of exactly two variables.
* One of the simplest forms of statistical analysis, used to find out if there is a relationship between two sets of values.

Examples: **scatter plots, pair-plots**

### Scatter Plot
* visualizes the bivariate relationships among several pairs of variables.
* The graph looks like a bunch of dots, but some of the graphs are a general shape or move in a general direction.
* To test the linear relationship between continuous variables Scatter plot is a good option. We can find out how one variable is changing w.r.t. another variable.

**1D scatter plot**

In [ ]:
for idx, feature in enumerate(list(hab.columns)[:-1]):
    plt.figure(figsize=(10,5))
    label = ['survived 5 years or longer','died within 5 year']
    plt.plot(class_1[feature], np.zeros_like(class_1[feature]), 'o')
    plt.plot(class_2[feature], np.zeros_like(class_2[feature]), 'o')
    plt.title("1-D scatter plot for {}".format(feature))
    plt.xlabel(feature)
    plt.legend(label)
    plt.show()

**Observations:**
* Of around 41-67 aged people died within 5 years.
* Below 37 aged people had chances to survive 5 years or longer than that.
* 1D scatter plot for operation year and +ve axillary nodes are not useful as they don't give much information.

**2D Scatter plot**

In [ ]:
import itertools
pairs=list(itertools.combinations(list(hab.columns)[:-1],2))
for p in pairs:
    label = ['survived 5 years or longer','died within 5 year']
    sns.set_style("whitegrid");
    sns.FacetGrid(hab, hue="Survival_status", size=5) \
       .map(plt.scatter, p[0], p[1]) 
    plt.legend(label)
    plt.show()

**Observations:**
* Classes are linearly inseparable in all the cases and doesn't convey much info.
* But we can draw little info from the plot between Positive axillary nodes and age, that the person having age between 50-60 and no of lymph detected between the range 0-3 have some chances to survive.


### PAIR PLOT
A pairs plot allows us to see both distribution of single variables and relationships between two variables.


In [ ]:
label = ['survived 5 years or longer','died within 5 year']
sns.set_style("whitegrid");
ax=sns.pairplot(hab, hue="Survival_status", vars = ["Age", "Operation_year", "Positive_axillary_nodes"],size=4);
plt.legend(label)
plt.show()
